# Extraction

In [259]:
# Import packages

import os
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Split

In [260]:
# Create list of all outputs files
files = [file for file in os.listdir(os.getcwd() + "/HEBBdata") if ".txt" in file]

In [261]:
# Split outputs files into visual and auditory
for file in files:
    if "AudioFirst" in file:
        with open(os.getcwd() + "/HEBBdata/" + file, "r") as src:
            with open(os.getcwd() + "/HEBBdataDivided/" + file[:-4] + " - Audio.txt", "w") as audiodest:
                for line in src.readlines()[:96]:
                    audiodest.write(line)
        with open(os.getcwd() + "/HEBBdata/" + file, "r") as src:
            with open(os.getcwd() + "/HEBBdataDivided/" + file[:-4] + " - Visual.txt", "w") as visualdest:
                for i, line in enumerate(src.readlines()[96:]):
                    visualdest.write(line.replace(str(math.ceil((i+1)/2)+48), str(math.ceil((i+1)/2))))
    elif "VisualFirst" in file:
        with open(os.getcwd() + "/HEBBdata/" + file, "r") as src:
            with open(os.getcwd() + "/HEBBdataDivided/" + file[:-4] + " - Visual.txt", "w") as visualdest:
                for line in src.readlines()[:96]:
                    visualdest.write(line)
        with open(os.getcwd() + "/HEBBdata/" + file, "r") as src:
            with open(os.getcwd() + "/HEBBdataDivided/" + file[:-4] + " - Audio.txt", "w") as audiodest:
                for i, line in enumerate(src.readlines()[96:]):
                    audiodest.write(line.replace(str(math.ceil((i+1)/2)+48), str(math.ceil((i+1)/2))))

## Total

In [262]:
files = [file for file in os.listdir(os.getcwd() + "\HEBBdataDivided") if ".txt" in file]

In [263]:
# Create empty lists for the data extraction

correct_occurrences = {i: [0 for j in range(9)] for i in range(12)}
correct_opportunities = {i: [0 for j in range(9)] for i in range(12)}

minus_occurrences = {i: [0 for j in range(8)] for i in range(12)}
minus_opportunities = {i: [0 for j in range(8)] for i in range(12)}
plus_occurrences = {i: [0 for j in range(8)] for i in range(12)}
plus_opportunities = {i: [0 for j in range(8)] for i in range(12)}

correct_slopes = []
error_slopes = []

repeated = [i for i in range(1, 49) if i % 4 == 0]

In [264]:
for file in files:
    with open(os.getcwd() + "\HEBBdataDivided\\" + file) as f:
        correct_count = [0 for i in range(9)]
        minus_count = [0 for i in range(8)]
        plus_count = [0 for i in range(8)]
        for line in f:
            splitted = line.strip().split(" ")
            if splitted[0] == "Clicked" and int(splitted[1].strip("():")) in repeated:
                answers = splitted[2:]
                for i in range(9):
                    correct_opportunities[correct_count[i]][i] += 1
                    if correct[i] == answers[i]:
                        correct_occurrences[correct_count[i]][i] += 1
                        correct_count[i] += 1
                for i in range(8):
                    minus_opportunities[minus_count[i]][i] += 1
                    plus_opportunities[plus_count[i]][i] += 1
                    if correct[i] == answers[i + 1]:
                        minus_occurrences[minus_count[i]][i] += 1
                        minus_count[i] += 1
                    if correct[i + 1] == answers[i]:
                        plus_occurrences[plus_count[i]][i] += 1
                        plus_count[i] += 1
            elif splitted[0] == "Said" and int(splitted[1].strip("():")) == 4:
                correct = splitted[2:]

In [265]:
with open("totalcorrect.txt", mode="w") as f:
    probabilities = []
    f.write("\t".join(["Serial_position"] + [str(i) for i in range(1, 5)]) + "\n")
    for i in range(9):
        f.write(str(i + 1))
        row_probability = []
        for j in range(1, 5):
            f.write("\t" + str(correct_occurrences[j][i] / correct_opportunities[j][i]))
            # append probabilities for j occurence
            row_probability.append(correct_occurrences[j][i] / correct_opportunities[j][i])
        f.write("\n")
        # append probabilities for i "row/serial position"
        probabilities.append(row_probability)
    # calculate slope for each "row/serial position"
    x = [1, 2, 3, 4]
    correct_slopes = []
    for i in range(9):
        slope, intercept = np.polyfit(x, probabilities[i], 1)
        correct_slopes.append(slope)
        print("Correct Slope " + str(i + 1) + ": " + str(slope))

Correct Slope 1: 0.037025401069518765
Correct Slope 2: 0.06895957769029291
Correct Slope 3: 0.023344371726038825
Correct Slope 4: 0.09743508811553833
Correct Slope 5: 0.029526976221406175
Correct Slope 6: 0.08504446640316204
Correct Slope 7: 0.03341147280234441
Correct Slope 8: 0.054290367874618155
Correct Slope 9: 0.08185639287612977


In [266]:
with open("totalerror.txt", mode="w") as f:
    probabilities = []
    f.write("\t".join(["Transposition"] + [str(i) for i in range(1, 5)]) + "\n")
    for i in range(8):
        f.write("T{} + 1".format(i + 1))
        row_probability = []
        for j in range(1, 5):
            if plus_opportunities[j][i]:
                result = str(plus_occurrences[j][i] / plus_opportunities[j][i])
                # append probabilities for j occurence
                row_probability.append(plus_occurrences[j][i] / plus_opportunities[j][i])
            else:
                result = "NaN"
            f.write("\t" + result)
        # append probabilities for i "row/transposition"
        probabilities.append(row_probability)
        row_probability = []
        f.write("\n")
        f.write("T{} - 1".format(i + 2))
        for j in range(1, 5):
            if minus_opportunities[j][i]:
                result = str(minus_occurrences[j][i] / minus_opportunities[j][i])
                # append probabilities for j occurence
                row_probability.append(minus_occurrences[j][i] / minus_opportunities[j][i])
            else:
                result = "NaN"
            f.write("\t" + result)
        # append probabilities for i "row/transposition"
        probabilities.append(row_probability)
        f.write("\n")
    # calculate slope for each "row/serial position"
    for i in range(16):
        slope, intercept = np.polyfit(list(range(1, len(probabilities[i]) + 1)), probabilities[i], 1)
        error_slopes.append(slope)
        print("Error Slope " + str(i + 1) + ": " + str(slope))

Error Slope 1: -0.0432098765432099
Error Slope 2: -0.036883223684210535
Error Slope 3: 0.06640729483282674
Error Slope 4: 0.07766483516483517
Error Slope 5: 0.09599551203590373
Error Slope 6: -0.025235487404162097
Error Slope 7: 0.01995160314579552
Error Slope 8: 0.2636559899940182
Error Slope 9: -0.038383239982766056
Error Slope 10: -0.04325608593953047
Error Slope 11: 0.03498734817813767
Error Slope 12: 0.05026471101254045
Error Slope 13: -0.023020707719038917
Error Slope 14: 0.028450486539019073
Error Slope 15: -0.012914805624483053
Error Slope 16: 0.011603490022543196


In [267]:
# Writes new file "correctwslopes.txt" with slope values
with open("totalcorrect.txt", "r") as src:
    with open("totalcorrectwslopes.txt", "w") as dest:
        for i, line in enumerate(src):
            if i == 0:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\tSlope"))
            else:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\t"+str(correct_slopes[i-1])))

In [268]:
# Writes new file "errorwslopes.txt" with slope values
with open("totalerror.txt", "r") as src:
    with open("totalerrorwslopes.txt", "w") as dest:
        for i, line in enumerate(src):
            if i == 0:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\tSlope"))
            else:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\t"+str(error_slopes[i-1])))

## Audio

In [269]:
# Create list of auditory files
files_audio = [file for file in os.listdir(os.getcwd() + "/HEBBdataDivided") if "- Audio" in file]

In [270]:
# Create empty lists for the data extraction

correct_occurrences = {i: [0 for j in range(9)] for i in range(12)}
correct_opportunities = {i: [0 for j in range(9)] for i in range(12)}

minus_occurrences = {i: [0 for j in range(8)] for i in range(12)}
minus_opportunities = {i: [0 for j in range(8)] for i in range(12)}
plus_occurrences = {i: [0 for j in range(8)] for i in range(12)}
plus_opportunities = {i: [0 for j in range(8)] for i in range(12)}

correct_slopes = []
error_slopes = []

repeated = [i for i in range(1, 49) if i % 4 == 0]

In [271]:
for file in files_audio:
    with open(os.getcwd() + "\HEBBdataDivided\\" + file) as f:
        correct_count = [0 for i in range(9)]
        minus_count = [0 for i in range(8)]
        plus_count = [0 for i in range(8)]
        for line in f:
            splitted = line.strip().split(" ")
            if splitted[0] == "Clicked" and int(splitted[1].strip("():")) in repeated:
                answers = splitted[2:]
                for i in range(9):
                    correct_opportunities[correct_count[i]][i] += 1
                    if correct[i] == answers[i]:
                        correct_occurrences[correct_count[i]][i] += 1
                        correct_count[i] += 1
                for i in range(8):
                    minus_opportunities[minus_count[i]][i] += 1
                    plus_opportunities[plus_count[i]][i] += 1
                    if correct[i] == answers[i + 1]:
                        minus_occurrences[minus_count[i]][i] += 1
                        minus_count[i] += 1
                    if correct[i + 1] == answers[i]:
                        plus_occurrences[plus_count[i]][i] += 1
                        plus_count[i] += 1
            elif splitted[0] == "Said" and int(splitted[1].strip("():")) == 4:
                correct = splitted[2:]

In [272]:
with open("audiocorrect.txt", mode="w") as f:
    probabilities = []
    f.write("\t".join(["Serial_position"] + [str(i) for i in range(1, 5)]) + "\n")
    for i in range(9):
        f.write(str(i + 1))
        row_probability = []
        for j in range(1, 5):
            f.write("\t" + str(correct_occurrences[j][i] / correct_opportunities[j][i]))
            # append probabilities for j occurence
            row_probability.append(correct_occurrences[j][i] / correct_opportunities[j][i])
        f.write("\n")
        # append probabilities for i "row/serial position"
        probabilities.append(row_probability)
    # calculate slope for each "row/serial position"
    x = [1, 2, 3, 4]
    correct_slopes = []
    for i in range(9):
        slope, intercept = np.polyfit(x, probabilities[i], 1)
        correct_slopes.append(slope)
        print("Correct Slope " + str(i + 1) + ": " + str(slope))

Correct Slope 1: 0.034044771018455204
Correct Slope 2: 0.0942442628859786
Correct Slope 3: 0.02856047536898591
Correct Slope 4: 0.10973725105304058
Correct Slope 5: 0.06497657306374012
Correct Slope 6: 0.16912206073496397
Correct Slope 7: 0.008265412748171375
Correct Slope 8: 0.0345971750966622
Correct Slope 9: 0.06232997703585946


In [273]:
with open("audioerror.txt", mode="w") as f:
    probabilities = []
    f.write("\t".join(["Transposition"] + [str(i) for i in range(1, 5)]) + "\n")
    for i in range(8):
        f.write("T{} + 1".format(i + 1))
        row_probability = []
        for j in range(1, 5):
            if plus_opportunities[j][i]:
                result = str(plus_occurrences[j][i] / plus_opportunities[j][i])
                # append probabilities for j occurence
                row_probability.append(plus_occurrences[j][i] / plus_opportunities[j][i])
            else:
                result = "NaN"
            f.write("\t" + result)
        # append probabilities for i "row/transposition"
        probabilities.append(row_probability)
        row_probability = []
        f.write("\n")
        f.write("T{} - 1".format(i + 2))
        for j in range(1, 5):
            if minus_opportunities[j][i]:
                result = str(minus_occurrences[j][i] / minus_opportunities[j][i])
                # append probabilities for j occurence
                row_probability.append(minus_occurrences[j][i] / minus_opportunities[j][i])
            else:
                result = "NaN"
            f.write("\t" + result)
        # append probabilities for i "row/transposition"
        probabilities.append(row_probability)
        f.write("\n")
    # calculate slope for each "row/serial position"
    for i in range(16):
        slope, intercept = np.polyfit(list(range(1, len(probabilities[i]) + 1)), probabilities[i], 1)
        error_slopes.append(slope)
        print("Error Slope " + str(i + 1) + ": " + str(slope))

Error Slope 1: -0.04285714285714288
Error Slope 2: -0.06307692307692309
Error Slope 3: 0.27733286085518555
Error Slope 4: 0.2782485875706215
Error Slope 5: 0.10047619047619052
Error Slope 6: -0.009523809523809516
Error Slope 7: 0.1214044943820225
Error Slope 8: 0.2503732981993852
Error Slope 9: -0.004166666666666681
Error Slope 10: -0.04622550497437445
Error Slope 11: 0.04725274725274725
Error Slope 12: 0.08311335403726709
Error Slope 13: -0.0201208031778854
Error Slope 14: 0.01707092907092907
Error Slope 15: 0.02550505050505052
Error Slope 16: 0.05845238095238096


In [274]:
# Writes new file "correctwslopes.txt" with slope values
with open("audiocorrect.txt", "r") as src:
    with open("audiocorrectwslopes.txt", "w") as dest:
        for i, line in enumerate(src):
            if i == 0:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\tSlope"))
            else:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\t"+str(correct_slopes[i-1])))

In [275]:
# Writes new file "errorwslopes.txt" with slope values
with open("audioerror.txt", "r") as src:
    with open("audioerrorwslopes.txt", "w") as dest:
        for i, line in enumerate(src):
            if i == 0:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\tSlope"))
            else:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\t"+str(error_slopes[i-1])))

## Visual 

In [276]:
# Create list of visual files
files_visual = [file for file in os.listdir(os.getcwd() + "/HEBBdataDivided") if "- Visual" in file]

In [277]:
# Create empty lists for the data extraction

correct_occurrences = {i: [0 for j in range(9)] for i in range(12)}
correct_opportunities = {i: [0 for j in range(9)] for i in range(12)}

minus_occurrences = {i: [0 for j in range(8)] for i in range(12)}
minus_opportunities = {i: [0 for j in range(8)] for i in range(12)}
plus_occurrences = {i: [0 for j in range(8)] for i in range(12)}
plus_opportunities = {i: [0 for j in range(8)] for i in range(12)}

correct_slopes = []
error_slopes = []

repeated = [i for i in range(1, 49) if i % 4 == 0]

In [278]:
for file in files_visual:
    with open(os.getcwd() + "\HEBBdataDivided\\" + file) as f:
        correct_count = [0 for i in range(9)]
        minus_count = [0 for i in range(8)]
        plus_count = [0 for i in range(8)]
        for line in f:
            splitted = line.strip().split(" ")
            if splitted[0] == "Clicked" and int(splitted[1].strip("():")) in repeated:
                answers = splitted[2:]
                for i in range(9):
                    correct_opportunities[correct_count[i]][i] += 1
                    if correct[i] == answers[i]:
                        correct_occurrences[correct_count[i]][i] += 1
                        correct_count[i] += 1
                for i in range(8):
                    minus_opportunities[minus_count[i]][i] += 1
                    plus_opportunities[plus_count[i]][i] += 1
                    if correct[i] == answers[i + 1]:
                        minus_occurrences[minus_count[i]][i] += 1
                        minus_count[i] += 1
                    if correct[i + 1] == answers[i]:
                        plus_occurrences[plus_count[i]][i] += 1
                        plus_count[i] += 1
            elif splitted[0] == "Said" and int(splitted[1].strip("():")) == 4:
                correct = splitted[2:]

In [279]:
with open("visualcorrect.txt", mode="w") as f:
    probabilities = []
    f.write("\t".join(["Serial_position"] + [str(i) for i in range(1, 5)]) + "\n")
    for i in range(9):
        f.write(str(i + 1))
        row_probability = []
        for j in range(1, 5):
            f.write("\t" + str(correct_occurrences[j][i] / correct_opportunities[j][i]))
            # append probabilities for j occurence
            row_probability.append(correct_occurrences[j][i] / correct_opportunities[j][i])
        f.write("\n")
        # append probabilities for i "row/serial position"
        probabilities.append(row_probability)
    # calculate slope for each "row/serial position"
    x = [1, 2, 3, 4]
    correct_slopes = []
    for i in range(9):
        slope, intercept = np.polyfit(x, probabilities[i], 1)
        correct_slopes.append(slope)
        print("Correct Slope " + str(i + 1) + ": " + str(slope))

Correct Slope 1: 0.0390909090909091
Correct Slope 2: 0.04560378062430044
Correct Slope 3: 0.019074494346233498
Correct Slope 4: 0.09035586277521766
Correct Slope 5: 0.015189459930313624
Correct Slope 6: 0.031663210738841
Correct Slope 7: 0.06378524534909037
Correct Slope 8: 0.07582117158204114
Correct Slope 9: 0.09030398186597659


In [280]:
with open("visualerror.txt", mode="w") as f:
    probabilities = []
    f.write("\t".join(["Transposition"] + [str(i) for i in range(1, 5)]) + "\n")
    for i in range(8):
        f.write("T{} + 1".format(i + 1))
        row_probability = []
        for j in range(1, 5):
            if plus_opportunities[j][i]:
                result = str(plus_occurrences[j][i] / plus_opportunities[j][i])
                # append probabilities for j occurence
                row_probability.append(plus_occurrences[j][i] / plus_opportunities[j][i])
            else:
                result = "NaN"
            f.write("\t" + result)
        # append probabilities for i "row/transposition"
        probabilities.append(row_probability)
        row_probability = []
        f.write("\n")
        f.write("T{} - 1".format(i + 2))
        for j in range(1, 5):
            if minus_opportunities[j][i]:
                result = str(minus_occurrences[j][i] / minus_opportunities[j][i])
                # append probabilities for j occurence
                row_probability.append(minus_occurrences[j][i] / minus_opportunities[j][i])
            else:
                result = "NaN"
            f.write("\t" + result)
        # append probabilities for i "row/transposition"
        probabilities.append(row_probability)
        f.write("\n")
    # calculate slope for each "row/serial position"
    for i in range(16):
        slope, intercept = np.polyfit(list(range(1, len(probabilities[i]) + 1)), probabilities[i], 1)
        error_slopes.append(slope)
        print("Error Slope " + str(i + 1) + ": " + str(slope))

Error Slope 1: -0.04347826086956525
Error Slope 2: -0.03174603174603178
Error Slope 3: 0.018565531475748197
Error Slope 4: 0.04464285714285715
Error Slope 5: -0.03787878787878791
Error Slope 6: -0.03037940379403796
Error Slope 7: -0.034173441734417345
Error Slope 8: -0.05232558139534885
Error Slope 9: -0.04450678733031673
Error Slope 10: -0.04081391736564152
Error Slope 11: 0.02484848484848485
Error Slope 12: 0.017344173441734424
Error Slope 13: 0.004356673531430795
Error Slope 14: 0.06469717497114756
Error Slope 15: -0.02671052631578946
Error Slope 16: -0.0051081602056115005


In [281]:
# Writes new file "correctwslopes.txt" with slope values
with open("visualcorrect.txt", "r") as src:
    with open("visualcorrectwslopes.txt", "w") as dest:
        for i, line in enumerate(src):
            if i == 0:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\tSlope"))
            else:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\t"+str(correct_slopes[i-1])))

In [282]:
# Writes new file "errorwslopes.txt" with slope values
with open("visualerror.txt", "r") as src:
    with open("visualerrorwslopes.txt", "w") as dest:
        for i, line in enumerate(src):
            if i == 0:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\tSlope"))
            else:
                dest.write("%s%s\n" % (line.rstrip("\n"), "\t"+str(error_slopes[i-1])))